In [ ]:
from datetime import datetime
import mysql.connector

# Connect to MySQL database
mycon = mysql.connector.connect(
    host="localhost",
    user="root",
    password="AshSQL@123",
    database="Streaming_Platform",
    connection_timeout=10
)
cursor = mycon.cursor()

def print_separator():
    print("-" * 50)

def signup():
    print("Signup for Streaming Platform")
    email = input("Enter Email: ").strip()
    username = input("Enter Username: ").strip()
    password = input("Enter Password: ").strip()
    dob = input("Enter DOB (YYYY-MM-DD): ").strip()
    account_role = "Free User"
    account_status = "Active"

    try:
        cursor.execute(
            "INSERT INTO USERS (Email, Username, Password, DOB, Account_Role, Account_Status) VALUES (%s, %s, %s, %s, %s, %s)",
            (email, username, password, dob, account_role, account_status)
        )
        mycon.commit()
        print("✅ Signup successful!")
    except mysql.connector.Error as err:
        print("Error:", err)
        mycon.rollback()

def login():
    print("Login to Streaming Platform")
    username = input("Enter Username: ").strip()
    password = input("Enter Password: ").strip()

    try:
        cursor.execute("SELECT User_ID, Username, Password FROM USERS WHERE Username = %s", (username,))
        user = cursor.fetchone()
        if user and password == user[2]:
            print("✅ Welcome back", user[1])
            try:
                cursor.execute("UPDATE USERS SET Last_Login = CURRENT_TIMESTAMP WHERE User_ID = %s", (user[0],))
                mycon.commit()
            except mysql.connector.Error as err:
                print("⚠️ Error updating login time:", err)
                mycon.rollback()
            return user[0]
        else:
            print("❌ Invalid username or password!")
            return None
    except mysql.connector.Error as err:
        print("Error:", err)
        mycon.rollback()
        return None

def manage_subscription(user_id):
    print_separator()
    print("Managing subscription for User", user_id)
    cursor.execute("SELECT Subscription_Plan, Subscription_Status FROM SUBSCRIPTIONS WHERE User_ID = %s", (user_id,))
    subscription = cursor.fetchone()

    if subscription:
        print(f"Current Subscription: {subscription[0]} - Status: {subscription[1]}")
        action = input("Do you want to update your subscription (Y/N)? ").strip().lower()
        if action == 'y':
            new_plan = input("Enter new subscription plan (Free, Standard, Premium): ").strip()
            try:
                cursor.execute("UPDATE SUBSCRIPTIONS SET Subscription_Plan = %s, Subscription_Status = 'Active' WHERE User_ID = %s",
                               (new_plan, user_id))
                mycon.commit()
                print("✅ Subscription updated successfully!")
            except mysql.connector.Error as err:
                print("❌ Error updating subscription:", err)
                mycon.rollback()
        else:
            print("No changes made.")
    else:
        print("You don't have an active subscription. Please subscribe to a plan.")
        new_plan = input("Enter subscription plan (Free, Standard, Premium): ").strip()
        start_date = datetime.today().date()
        end_date = datetime.today().replace(year=datetime.today().year + 1).date()
        try:
            cursor.execute("INSERT INTO SUBSCRIPTIONS (User_ID, Subscription_Plan, Start_Date, End_Date, Subscription_Status) VALUES (%s, %s, %s, %s, %s)",
                           (user_id, new_plan, start_date, end_date, "Active"))
            mycon.commit()
            print("✅ Subscription created successfully!")
        except mysql.connector.Error as err:
            print("❌ Error creating subscription:", err)
            mycon.rollback()

def add_to_watchlist(user_id):
    print_separator()
    print(f"Managing watchlist for User {user_id}")
    content_id = int(input("Enter Content ID to add to watchlist: ").strip())

    cursor.execute("SELECT Content_ID FROM PLATFORM_CONTENT WHERE Content_ID = %s", (content_id,))
    if not cursor.fetchone():
        print("❌ Error: Content does not exist.")
        return

    try:
        cursor.execute("INSERT INTO WATCHLIST (User_ID, Content_ID) VALUES (%s, %s)", (user_id, content_id))
        mycon.commit()
        print("✅ Content added to your watchlist!")
    except mysql.connector.Error as err:
        print("❌ Error adding to watchlist:", err)
        mycon.rollback()

def rate_content(user_id):
    print_separator()
    print(f"Rating and reviewing content for User {user_id}")
    content_id = int(input("Enter Content ID you want to rate and review: ").strip())

    cursor.execute("SELECT Content_ID FROM PLATFORM_CONTENT WHERE Content_ID = %s", (content_id,))
    if not cursor.fetchone():
        print("❌ Error: Content does not exist.")
        return

    rating = int(input("Enter your rating (1-5): ").strip())
    review = input("Enter your review: ").strip()

    try:
        cursor.execute(
            "INSERT INTO USER_INTERACTION (User_ID, Content_ID, Rating, Review) VALUES (%s, %s, %s, %s)",
            (user_id, content_id, rating, review)
        )
        mycon.commit()
        print("✅ Rating and review added!")
    except mysql.connector.Error as err:
        if "Lock wait timeout exceeded" in str(err):
            print("❌ Lock wait timeout exceeded. Retrying failed.")
        else:
            print("❌ Error while rating:", err)
        mycon.rollback()

def delete_account(user_id):
    print_separator()
    confirm = input("Are you sure you want to delete your account? (Y/N): ").strip().lower()
    if confirm != 'y':
        print("Account deletion cancelled.")
        return

    try:
        cursor.execute("DELETE FROM USERS WHERE User_ID = %s", (user_id,))
        mycon.commit()
        print("✅ Your account has been deleted. Goodbye!")
    except mysql.connector.Error as err:
        print("❌ Error deleting account:", err)
        mycon.rollback()

def delete_content(user_id, role):
    print_separator()
    if role.lower() != "administrator":
        print("❌ Only administrators can delete content.")
        return

    content_id = int(input("Enter Content ID to delete: ").strip())
    cursor.execute("SELECT Content_ID FROM PLATFORM_CONTENT WHERE Content_ID = %s", (content_id,))
    if not cursor.fetchone():
        print("❌ Error: Content does not exist.")
        return

    try:
        cursor.execute("DELETE FROM PLATFORM_CONTENT WHERE Content_ID = %s", (content_id,))
        mycon.commit()
        print("✅ Content deleted successfully!")
    except mysql.connector.Error as err:
        print("❌ Error deleting content:", err)
        mycon.rollback()


def main():
    while True:
        print_separator()
        print("Welcome to the Streaming Platform!")
        print("1. Sign Up")
        print("2. Login")
        print("3. Exit")
        choice = input("Choose an option (1/2/3): ").strip()

        if choice == "1":
            signup()
        elif choice == "2":
            user_id = login()
            if user_id:
                while True:
                    print_separator()
                    print("1. Manage Subscription")
                    print("2. Add Content to Watchlist")
                    print("3. Rate and Review Content")
                    print("4. Logout")
                    action = input("Choose an option: ").strip()

                    if action == "1":
                        manage_subscription(user_id)
                    elif action == "2":
                        add_to_watchlist(user_id)
                    elif action == "3":
                        rate_content(user_id)
                    elif action == "4":
                        print("Logging out...")
                        break
                    else:
                        print("Invalid option. Try again.")
        elif choice == "3":
            print("Goodbye!")
            break
        else:
            print("Invalid option. Try again.")

# Run the program
if __name__ == "__main__":
    try:
        main()
    finally:
        cursor.close()
        mycon.close()


--------------------------------------------------
Welcome to the Streaming Platform!
1. Sign Up
2. Login
3. Exit


Choose an option (1/2/3):  2


Login to Streaming Platform


Enter Username:  noballs
Enter Password:  12345


✅ Welcome back noballs
--------------------------------------------------
1. Manage Subscription
2. Add Content to Watchlist
3. Rate and Review Content
4. Logout


Choose an option:  4


Logging out...
--------------------------------------------------
Welcome to the Streaming Platform!
1. Sign Up
2. Login
3. Exit


Choose an option (1/2/3):  3


Goodbye!
